In [ ]:
import pandas as pd
import numpy as np

import pickle

import time

In [ ]:
def current_milli_time():
    return round(time.time() * 1000)

names = {
    "imputer": f"models/imputer.pickle",
    "tpot_death": f"models/tpot_death.pickle",
    "tpot_sev": f"models/tpot_sev.pickle",
    "tpot_dead": f"models/tpot_dead.pickle",
    "tpot_alive": f"models/tpot_alive.pickle",
}
cols = [
    "sex",
    "ill_to_hosp",
    "age",
    "temp",
    "weakness",
    "smell_taste_loss",
    "chd",
    "aks",
    "cvd",
    "imv_days",
    "pcr_1",
    "leuk_1",
    "neut_1",
    "l_1",
    "ldh_1",
    "crp_1",
    "fer_1",
    "d_dym_1",
    "il6_1",
    "creat_1",
    "glu_1",
    "pct",
    "news_1",
    "resp_fail_5",
]
filename = "data2.txt"



imputer = pickle.load(open(names["imputer"], "rb"))
tpot_death = pickle.load(open(names["tpot_death"], "rb"))
tpot_sev = pickle.load(open(names["tpot_sev"], "rb"))
tpot_dead = pickle.load(open(names["tpot_dead"], "rb"))
tpot_alive = pickle.load(open(names["tpot_alive"], "rb"))

data = pd.read_csv(filename, names=cols)
data = pd.DataFrame(imputer.transform(data), columns=data.columns, index=data.index)
death_rate = tpot_death.predict_proba(data)[0][1]
death = tpot_death.predict(data)[0]
sev_rate = max(tpot_sev.predict_proba(data)[0])
sev = tpot_sev.predict(data)[0]
days = np.nan
if death == 0:
    days = tpot_alive.predict(data)[0]
else:
    days = tpot_death.predict(data)[0]
    
answer = np.array([[death_rate, death, sev_rate, sev, days]])
np.savetxt(f"answer{current_milli_time()}.csv", answer, delimiter=",")

In [ ]:
data = [death_rate, death, sev_rate, sev, days]
names = ['Вероятность смерти', 'умрет?', 'вероятность тяжести', 'тяжесть', 'предполагаемые дни']
if data[0] > 0.5:
    print(f"It is most likely that patient will die.")
else:
    print(f"It is most likely that patient will survive.")
print(f"Probability of death = {data[0]*100}%")
if data[3]==1:
    print(f"With probability of {data[2]*100}% severity will be light.")
elif data[3]==2:
    print(f"With probability of {data[2]*100}% severity will be middle.")
else:
    print(f"With probability of {data[2]*100}% severity will be hard.")
print(f'Approximate time until discharge = {data[4]}.')